Firstly, we import necessarcy packages, eg. MatCalc, MatGL and Pymatgen.

In [1]:
from __future__ import annotations

import json
import random
import warnings

import matgl
import numpy as np
import pandas as pd
from matcalc.elasticity import ElasticityCalc
from matcalc.phonon import PhononCalc
from matgl.ext.ase import PESCalculator
from pymatgen.core import Structure

# To suppress warnings for clearer output
warnings.simplefilter("ignore")

We leverage the physical constants provided by SciPy to convert stress units, ensuring compatibility with the ASE calculator.

In [2]:
from scipy import constants

eVA3ToGPa = constants.e / (constants.angstrom) ** 3 / constants.giga

Next, I will show you how to run MatCalc-Bench. Here, TensorNet-MatPES-PBE-v2025.1-PES and M3GNet-MatPES-PBE-v2025.1-PES are used to perform elastic modulus calculations. The approach is similar for other models and other benchmarks. For demonstration purposes only, I randomly sample 1% of the entire test dataset.

In [3]:
all_results = {}

for model_name in [
    "M3GNet-MatPES-PBE-v2025.1-PES",
    "TensorNet-MatPES-PBE-v2025.1-PES",
]:
    potential = matgl.load_model(model_name)
    calculator = PESCalculator(potential, stress_weight=1 / eVA3ToGPa)

    with open("MatCalc-Bench_data/elastic.json") as f:
        entries = json.load(f)

    random.seed(2025)
    sampled_entries = random.sample(entries, len(entries) // 100)

    elastc_calc = ElasticityCalc(calculator, fmax=0.05, relax_structure=True)

    bulk_modulus_ae = []
    shear_modulus_ae = []

    for entry in sampled_entries:
        mp_id = entry["mp_id"]
        if mp_id not in all_results:
            all_results[mp_id] = {
                "mp_id": mp_id,
                "formula": entry["formula"],
                "K-DFT": entry["bulk_modulus_vrh"],
                "G-DFT": entry["shear_modulus_vrh"],
                "K-M3GNet-MatPES": None,
                "G-M3GNet-MatPES": None,
                "K-TensorNet-MatPES": None,
                "G-TensorNet-MatPES": None,
            }
        structure = Structure.from_dict(entry["structure"])
        properties = elastc_calc.calc(structure)

        predicted_bulk = properties["bulk_modulus_vrh"] * eVA3ToGPa
        predicted_shear = properties["shear_modulus_vrh"] * eVA3ToGPa

        if model_name.startswith("M3GNet"):
            all_results[mp_id]["K-M3GNet-MatPES"] = predicted_bulk
            all_results[mp_id]["G-M3GNet-MatPES"] = predicted_shear
        elif model_name.startswith("TensorNet"):
            all_results[mp_id]["K-TensorNet-MatPES"] = predicted_bulk
            all_results[mp_id]["G-TensorNet-MatPES"] = predicted_shear

df = pd.DataFrame(list(all_results.values()))

df = df[
    [
        "mp_id",
        "formula",
        "K-DFT",
        "G-DFT",
        "K-M3GNet-MatPES",
        "G-M3GNet-MatPES",
        "K-TensorNet-MatPES",
        "G-TensorNet-MatPES",
    ]
]

df.to_csv("MatCalc-Bench_data/elastic.csv", index=False)

You can certainly perform the full benchmark by slightly modifying the codes provided below, but please be aware that the whole process is considerably time consuming. Therefore, I recommend submitting separate jobs to your cluster for every material—or every few materials, depending on your resource availability and workflow preferences.

Finally, I will demonstrate how to do t-test to rigourously evaluate whether the means between two distributions are equal. For reference, this is an explanation:
https://www.jmp.com/en/statistics-knowledge-portal/t-test/two-sample-t-test

In [4]:
from scipy.stats import ttest_ind, ttest_rel

Read the data obtained from the above workflow. Here I used all the elastic moduli calculated by myself.

In [5]:
df = pd.read_csv("MatCalc-Bench_data/my_elastic.csv")
df = df[df["K-DFT"] < 500]
df = df[df["G-DFT"] < 500]
df = df.dropna()

In [6]:
for col in df.columns:
    if (col.startswith("K") or col.startswith("G")) and not col.endswith("DFT"):
        prop, architecture, dataset = col.split("-")
        df[f"AE {prop}-{architecture}-{dataset}"] = np.abs(df[col] - df[f"{prop}-DFT"])

In [7]:
def analyze_stats(prop):
    data = df[[c for c in df.columns if c.startswith(f"AE {prop}")]]
    means = data.describe().loc["mean"]
    best_model = means.idxmin()
    stats = []
    for col in data.columns:
        _, architecture, dataset = col.split("-")
        result_ind = ttest_ind(data[col], data[best_model], equal_var=False)
        result_rel = ttest_rel(data[col], data[best_model])

        stats.append([dataset, architecture, data[col].mean(), data[col].std(), result_ind.pvalue, result_rel.pvalue])
    stats = pd.DataFrame(
        stats,
        columns=["Dataset", "Architecture", f"{prop} MAE", f"{prop} STDAE", f"{prop} p_diff_ind", f"{prop} p_diff_rel"],
    )
    stats[f"{prop} sig_diff_ind"] = stats[f"{prop} p_diff_ind"] < 0.05
    stats[f"{prop} sig_diff_rel"] = stats[f"{prop} p_diff_rel"] < 0.05
    return stats

In [8]:
k_stats = analyze_stats("K")
g_stats = analyze_stats("G")

In [9]:
index_cols = ["Dataset", "Architecture"]
all_stats = k_stats.set_index(index_cols).join(g_stats.set_index(index_cols), on=index_cols)

In [10]:
all_stats

K MAE    K STDAE  K p_diff_ind  K p_diff_rel  \
Dataset Architecture                                                     
MatPES  M3GNet        26.109960  30.648437  7.060440e-66  1.522520e-92   
        TensorNet     18.022629  21.680635  1.367447e-06  1.111028e-15   
        CHGNet        23.726683  27.884163  3.367077e-45  2.393578e-70   
MPF     M3GNet        21.356872  27.474171  5.023621e-24  1.097924e-40   
        TensorNet     24.424730  34.246278  5.756311e-41  3.373360e-68   
MPtrj   CHGNet        17.447911  59.969943  7.213025e-02  5.133258e-02   
OMat24  TensorNet     15.613306  22.603645  1.000000e+00           NaN   

                      K sig_diff_ind  K sig_diff_rel      G MAE     G STDAE  \
Dataset Architecture                                                          
MatPES  M3GNet                  True            True  25.299590  233.879266   
        TensorNet               True            True  14.815028   38.371662   
        CHGNet                  True            True  20.610260  144.185286   
MPF     M3GNet                  True            True  40.967879  121.661666   
        TensorNet               True            True  36.008103   55.516632   
MPtrj   CHGNet                 False           False  29.591159   34.994110   
OMat24  TensorNet              False           False  15.496331   36.955297   

                      G p_diff_ind   G p_diff_rel  G sig_diff_ind  \
Dataset Architecture                                                
MatPES  M3GNet        5.467365e-03   3.433715e-03            True   
        TensorNet     1.000000e+00            NaN           False   
        CHGNet        1.470508e-02   1.263102e-02            True   
MPF     M3GNet        2.421706e-37   7.204788e-40            True   
        TensorNet     2.166871e-84  7.324933e-100            True   
MPtrj   CHGNet        4.301938e-70   7.462499e-94            True   
OMat24  TensorNet     4.216771e-01   2.603470e-01           False   

                      G sig_diff_rel  
Dataset Architecture                  
MatPES  M3GNet                  True  
        TensorNet              False  
        CHGNet                  True  
MPF     M3GNet                  True  
        TensorNet               True  
MPtrj   CHGNet                  True  
OMat24  TensorNet              False